#### Imports

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from IPython.display import display
import scipy.stats as stats
import matplotlib.pyplot as plt

In [3]:
# set the path to the directory containing the folder 'DATA'
path_start = '/Users/markoshea/Desktop/Working Directories/Datasets/rep_files/DATA/'

#### 1: From create.do:

In [27]:
#1) from create.do:
indshares = pd.read_stata(path_start + 'indshares.dta')
otheroutcomes = pd.read_stata(path_start + 'otheroutcomes.dta')

## instruments
for lag in [0, 10, 20, 30, 40]:
    globals()[f'spred{lag}'] = pd.read_stata(f"{path_start}spred{lag}.dta")

In [5]:
#industry shares in a given state/year for 51 states (including DC) for 
# 1949, 1959, 1969, 1979, 1989, 1999, 2009
indshares

,stfips,year,IND1,IND2,IND3,IND4,IND5,IND6,IND7,IND8,IND9,IND10,IND11,IND12,IND13,IND14
0,Alabama,1949,0.172709,0.024749,0.045725,0.180276,0.054233,0.017108,0.022326,0.107970,0.018488,0.016899,0.087342,0.004175,0.080683,0.164974
1,Alabama,1959,0.111314,0.011004,0.073814,0.265247,0.042504,0.023402,0.025715,0.123332,0.026607,0.015810,0.096337,0.004304,0.094697,0.052093
2,Alabama,1969,0.040129,0.007196,0.065498,0.262638,0.040775,0.026937,0.035701,0.128875,0.031919,0.019742,0.072509,0.004244,0.140590,0.058579
3,Alabama,1979,0.027247,0.011601,0.069292,0.259738,0.037241,0.035799,0.041242,0.128664,0.045570,0.028050,0.037821,0.004938,0.192647,0.067790
4,Alabama,1989,0.023440,0.007343,0.073146,0.231570,0.039340,0.034689,0.041357,0.132605,0.050093,0.039904,0.030652,0.007298,0.222857,0.065707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Wyoming,1969,0.086842,0.094737,0.062281,0.060526,0.057018,0.029825,0.029825,0.150877,0.029825,0.015789,0.061404,0.006140,0.168421,0.067544
351,Wyoming,1979,0.056466,0.138755,0.099004,0.058181,0.052716,0.041466,0.026572,0.142505,0.041359,0.027965,0.038359,0.006000,0.199186,0.060966
352,Wyoming,1989,0.060362,0.078945,0.079741,0.060259,0.052370,0.036262,0.024170,0.150066,0.041073,0.035229,0.035170,0.011895,0.255335,0.079123
353,Wyoming,1999,0.044245,0.068983,0.089407,0.056485,0.052944,0.032799,0.023262,0.136927,0.045476,0.041123,0.039703,0.010389,0.273378,0.084878


Explanation of column names for `indshares.dta`:

- **IND1** → `ind1990` between **2 and 32** (`1 < ind1990 < 33`)  
- **IND2** → `ind1990` between **40 and 50** (`39 < ind1990 < 51`)  
- **IND3** → `ind1990` is **exactly 60** (`ind1990 == 60`)  
- **IND4** → `ind1990` between **62 and 399** (`61 < ind1990 < 400`)  
- **IND5** → `ind1990` between **400 and 432** (`399 < ind1990 < 433`)  
- **IND6** → `ind1990` between **440 and 472** (`439 < ind1990 <= 472`)  
- **IND7** → `ind1990` between **500 and 571** (`499 < ind1990 < 572`)  
- **IND8** → `ind1990` between **580 and 691** (`579 < ind1990 < 692`)  
- **IND9** → `ind1990` between **700 and 712** (`699 < ind1990 < 713`)  
- **IND10** → `ind1990` between **714 and 760** (`713 < ind1990 < 761`)  
- **IND11** → `ind1990` between **761 and 791** (`760 < ind1990 < 792`)  
- **IND12** → `ind1990` between **800 and 810** (`799 < ind1990 < 811`)  
- **IND13** → `ind1990` between **812 and 893** (`811 < ind1990 < 894`)  
- **IND14** → `ind1990` greater than **893** (`893 < ind1990`)  

Therefore, from the 1990 Census Industry Codes (`ind1990`):

- **IND1** - Agriculture, Forestry, Fishing, Hunting  
- **IND2** - Mining (includes Oil and Gas Extraction)  
- **IND3** - Utilities  
- **IND4** - Construction and Manufacturing  
- **IND5** - Wholesale Trade  
- **IND6** - Retail Trade  
- **IND7** - Transportation and Warehousing  
- **IND8** - Information  
- **IND9** - Finance and Insurance  
- **IND10** - Real Estate and Rental Leasing  
- **IND11** - Professional, Scientific, and Technical Services  
- **IND12** - Educational Services  
- **IND13** - Health Care and Social Assistance  
- **IND14** - Arts, Entertainment, Recreation, Accommodation, and Food Services  

IND1 through IND14 represent the share of employment or economic activity attributed to specific industry sectors within each state and year. Source: https://www.census.gov/content/dam/Census/about/about-the-bureau/adrm/data-linkage/HUDMetadata/HUD-MTO/IndustryOccupationCodes/1990%20Census_SIC%20codes.pdf

In [6]:
# state/year data for 51 states (including DC) for 1979, 1989, 1999, 2009
otheroutcomes

,year,stfips,earn,age2,age3,age4,age5,age6,age7,age8,...,work,tothrs,tothrs2,work_1,work_2,work_3,wage_1,wage_2,wage_3,nocollegedegree
0,1979,Alabama,1.690472e+10,679820.000000,511740.000000,392000.000000,387360.000000,321620.0,206840.000000,68240.000000,...,1.643700e+06,2.888217e+09,2.890617e+09,0.797778,0.705040,0.236225,5.505393,6.442639,5.643524,0.885083
1,1989,Alabama,3.420734e+10,620528.000000,636212.000000,505745.000000,377119.000000,348629.0,235893.000000,102440.000000,...,1.900391e+06,3.416881e+09,3.421085e+09,0.833035,0.772300,0.244756,8.849933,11.650237,9.975562,0.852752
2,1999,Alabama,5.817407e+10,613166.000000,646174.000000,657905.000000,506400.000000,358763.0,266242.000000,122454.000000,...,2.112515e+06,3.963541e+09,3.967632e+09,0.822135,0.769441,0.235339,12.516436,16.695501,16.417269,0.820616
3,2009,Alabama,7.556565e+10,646947.666667,599157.333333,658938.000000,650100.000000,485716.0,285698.666667,141800.000000,...,2.187780e+06,4.013475e+09,0.000000e+00,0.784339,0.736300,0.262732,15.503610,21.167253,21.680973,0.790927
4,1979,Alaska,3.359198e+09,93520.000000,74880.000000,41820.000000,28340.000000,13100.0,5240.000000,1320.000000,...,2.091200e+05,3.625095e+08,3.625796e+08,0.863777,0.796465,0.392533,8.392041,11.248151,10.394285,0.819258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2009,Wisconsin,1.095161e+11,765376.333333,688255.000000,822940.000000,826386.333333,543151.0,315054.000000,192443.000000,...,3.018889e+06,5.484949e+09,0.000000e+00,0.874358,0.850468,0.316022,16.752171,22.357214,20.937025,0.751681
200,1979,Wyoming,2.835783e+09,101500.000000,69820.000000,43340.000000,39840.000000,29520.0,15640.000000,5980.000000,...,2.339600e+05,4.386603e+08,4.390665e+08,0.867966,0.808608,0.344960,6.491745,6.598011,5.581069,0.845181
201,1989,Wyoming,3.985084e+09,61218.000000,81944.000000,57961.000000,38125.000000,33151.0,21583.000000,8955.000000,...,2.283100e+05,4.117203e+08,4.120346e+08,0.881393,0.845930,0.319587,8.849796,11.153350,8.134697,0.828206
202,1999,Wyoming,6.583203e+09,65767.000000,65798.000000,82598.000000,59595.000000,34821.0,27393.000000,12534.000000,...,2.629720e+05,4.957410e+08,4.959175e+08,0.897541,0.855464,0.301618,11.270205,15.082357,13.003825,0.803739


Explanation of column names for `otheroutcomes.dta`:

- **year** – The specific year the data pertains to.  
- **stfips** – The State FIPS Code, a unique two-digit numerical identifier assigned to each U.S. state and territory.  
- **earn** – The total sum of **wage earnings** in the state-year.  
- **age2** – The number of people **aged 20-29** in the state-year.  
- **age3** – The number of people **aged 30-39** in the state-year.  
- **age4** – The number of people **aged 40-49** in the state-year.  
- **age5** – The number of people **aged 50-59** in the state-year.  
- **age6** – The number of people **aged 60-69** in the state-year.  
- **age7** – The number of people **aged 70-79** in the state-year.  
- **age8** – The number of people **aged 80-89** in the state-year.  
- **age9** – The number of people **aged 90+** in the state-year.  
- **cnn** – The total number of individuals in the dataset for a given state-year. 
- **work** – The total number of individuals who are **working** in the state-year.  
- **tothrs** – The total **predicted** number of hours worked in the state-year (based on `wkswork1p`).  
- **tothrs2** – The total number of hours worked in the state-year using the **original** work weeks variable (`wkswork1`).  
- **work_1** – The **average workforce participation rate** for people **under 40** in the state-year.  
- **work_2** – The **average workforce participation rate** for people **aged 40-59** in the state-year.  
- **work_3** – The **average workforce participation rate** for people **aged 60+** in the state-year.  
- **wage_1** – The **average wage** for people **under 40** in the state-year.  
- **wage_2** – The **average wage** for people **aged 40-59** in the state-year.  
- **wage_3** – The **average wage** for people **aged 60+** in the state-year.  
- **nocollegedegree** – The proportion of people **without a college degree** in the state-year. 

In [28]:
spred10

,year,stfips,pred10num1,pred10num2,pred10num3,pred10num4,pred10num5,pred10num6,pred10num7,pred10num8,pred10num9,pred10all
0,1909,Alabama,585514.625000,475631.250000,361204.687500,235596.015625,142190.437500,93483.500000,41196.300781,10507.333984,1198.336914,1.361008e+06
1,1919,Alabama,624777.937500,551667.312500,403153.093750,280060.625000,201806.718750,105099.570312,53081.058594,13601.478516,1731.952881,1.610202e+06
2,1929,Alabama,630614.062500,569343.000000,438423.093750,303692.531250,229721.796875,144955.906250,55840.828125,15719.131836,1916.249146,1.759612e+06
3,1939,Alabama,635996.625000,609385.437500,520268.906250,404853.312500,268814.250000,178646.468750,90359.695312,20662.902344,2432.446533,2.095423e+06
4,1949,Alabama,717758.812500,559893.812500,487697.906250,419747.218750,332616.375000,203780.093750,122381.890625,32355.273438,2955.300781,2.161428e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
552,1969,Wyoming,68921.937500,67428.929688,49119.144531,35614.238281,37772.496094,31126.798828,18188.546875,7349.518555,1015.731506,2.476154e+05
553,1979,Wyoming,90429.578125,79071.296875,105248.703125,69798.335938,41910.062500,35463.785156,22033.177734,7957.147949,1346.731445,3.628292e+05
554,1989,Wyoming,84826.859375,80066.257812,66677.296875,83754.953125,56493.292969,35303.464844,26155.691406,12013.445312,2136.564453,3.626010e+05
555,1999,Wyoming,69256.554688,82967.632812,69295.281250,66264.539062,80096.343750,56890.628906,28794.392578,15875.021484,3211.578125,4.033954e+05


Explanation of column names for `spred10`:
- **year** – The specific year the data pertains to.  
- **stfips** – The State FIPS Code, a unique two-digit numerical identifier assigned to each U.S. state and territory.  
- **pred10num_k(s, t)** = predicted number of people in state s, t + 10, who were in age bin k at year t.
- **pred10all** = total predicted population aged 10+ at t + 10

Note that spred10 through spred40 follow the same structure as spred0 in column names and data types. Note that these are lagged so that spred10.dta uses age structures from year - 10, spred20.dta uses from year - 20, and so on.

#### 2: From create_bpl.do:

In [18]:
#2) from create_bpl.do (same as create.do, but uses place of residence in the 
# previous year instead of current year for Table A10):
otheroutcomes_bpl = pd.read_stata(path_start + 'otheroutcomes_bpl.dta')
spred10_bpl = pd.read_stata(path_start + 'spred10_bpl.dta')

In [19]:
otheroutcomes_bpl

,year,stfips,poploc,earn,age2,age3,age4,age5,age6,age7,...,work,tothrs,tothrs2,work_1,work_2,work_3,wage_1,wage_2,wage_3,nocollegedegree
0,1999,Alabama,187069.0,5.817407e+10,613166.000000,646174.000000,657905.000000,506400.000000,358763.000000,266242.000000,...,2.112515e+06,3.964440e+09,3.967632e+09,0.822135,0.769441,0.235339,12.513549,16.692030,16.411985,0.820616
1,2009,Alabama,907040.0,7.530679e+10,643299.000000,592984.000000,653923.333333,649360.666667,483142.666667,285056.333333,...,2.176369e+06,3.995055e+09,0.000000e+00,0.786460,0.735904,0.262070,15.494834,21.212828,21.705198,0.790537
2,1999,Alaska,24204.0,1.050963e+10,84830.000000,101764.000000,114001.000000,68161.000000,28965.000000,15967.000000,...,3.376030e+05,6.115678e+08,6.123156e+08,0.877879,0.855414,0.347776,14.340936,19.960657,17.918724,0.772584
3,2009,Alaska,158889.0,1.702236e+10,121460.666667,100089.333333,109808.666667,107655.000000,59269.666667,24873.333333,...,4.117103e+05,7.589605e+08,0.000000e+00,0.854132,0.840108,0.417152,18.879530,25.659828,23.191452,0.752796
4,1999,Arizona,231472.0,7.357752e+10,735670.000000,767315.000000,714058.000000,533157.000000,392063.000000,319105.000000,...,2.452022e+06,4.588466e+09,4.590817e+09,0.825570,0.807066,0.234716,13.886732,18.312510,17.363073,0.779731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2009,West Virginia,388206.0,2.727130e+10,228129.000000,227029.000000,250484.000000,279289.000000,215181.333333,127266.333333,...,8.226040e+05,1.518204e+09,0.000000e+00,0.763727,0.711863,0.232535,15.388000,19.812794,18.987354,0.827766
98,1999,Wisconsin,190513.0,8.535540e+10,689014.000000,806081.000000,839041.000000,587808.000000,390646.000000,319387.000000,...,2.880923e+06,5.453806e+09,5.460429e+09,0.911555,0.883469,0.282562,13.723597,17.517805,16.010962,0.786454
99,2009,Wisconsin,929323.0,1.094912e+11,763121.000000,688339.000000,822197.000000,825634.333333,543749.666667,315624.333333,...,3.019775e+06,5.492714e+09,0.000000e+00,0.876412,0.851011,0.315698,16.698347,22.343899,20.902395,0.750900
100,1999,Wyoming,13143.0,6.583203e+09,65767.000000,65798.000000,82598.000000,59595.000000,34821.000000,27393.000000,...,2.629720e+05,4.958444e+08,4.959175e+08,0.897541,0.855464,0.301618,11.267901,15.079386,12.999712,0.803739


Explanation of column names for `otheroutcomes_bpl.dta`:

- **year** – The specific year the data pertains to.  
- **stfips** – The State FIPS Code, a unique two-digit numerical identifier assigned to each U.S. state and territory.  
- **poploc** – The total population in the state-year (based on past residence, `migplac1`).  
- **earn** – The total sum of **wage earnings** in the state-year.  
- **age2** – The number of people **aged 20-29** in the state-year.  
- **age3** – The number of people **aged 30-39** in the state-year.  
- **age4** – The number of people **aged 40-49** in the state-year.  
- **age5** – The number of people **aged 50-59** in the state-year.  
- **age6** – The number of people **aged 60-69** in the state-year.  
- **age7** – The number of people **aged 70-79** in the state-year.  
- **age8** – The number of people **aged 80-89** in the state-year.  
- **age9** – The number of people **aged 90+** in the state-year.  
- **cnn** – The total number of individuals in the dataset for a given state-year. 
- **work** – The total number of individuals who are **working** in the state-year.  
- **tothrs** – The total **predicted** number of hours worked in the state-year (based on `wkswork1p`).  
- **tothrs2** – The total number of hours worked in the state-year using the **original** work weeks variable (`wkswork1`).  
- **work_1** – The **average workforce participation rate** for people **under 40** in the state-year.  
- **work_2** – The **average workforce participation rate** for people **aged 40-59** in the state-year.  
- **work_3** – The **average workforce participation rate** for people **aged 60+** in the state-year.  
- **wage_1** – The **average wage** for people **under 40** in the state-year.  
- **wage_2** – The **average wage** for people **aged 40-59** in the state-year.  
- **wage_3** – The **average wage** for people **aged 60+** in the state-year.  
- **nocollegedegree** – The proportion of people **without a college degree** in the state-year.  

In [59]:
spred10_bpl

,year,stfips,pred10num1,pred10num2,pred10num3,pred10num4,pred10num5,pred10num6,pred10num7,pred10num8,pred10num9,pred10all
0,1999,Alabama,654098.312500,667805.875000,640890.812500,647923.125000,635990.875000,481486.593750,295028.312500,153574.593750,31385.167969,3.554085e+06
1,2009,Alabama,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2,1999,Alaska,109673.125000,107681.382812,88638.320312,102028.460938,110236.070312,64926.941406,24066.109375,9333.791016,1456.035156,5.083671e+05
3,2009,Alaska,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
4,1999,Arizona,822689.687500,773809.250000,769050.187500,770214.125000,690369.312500,506857.218750,321625.031250,183462.656250,34667.667969,4.050056e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
97,2009,West Virginia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
98,1999,Wisconsin,771411.750000,836334.437500,720074.062500,807360.562500,811140.812500,558836.625000,320810.093750,183317.218750,43436.691406,4.281310e+06
99,2009,Wisconsin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
100,1999,Wyoming,68865.195312,81939.015625,68756.570312,65971.382812,79836.851562,56698.152344,28710.996094,15844.160156,3207.163086,4.009643e+05


Explanation of column names for `spred10_bpl.dta`:

- **year** – The specific year the data pertains to.  
- **stfips** – The State FIPS Code, a unique two-digit numerical identifier assigned to each U.S. state and territory.  
- **pred10num1** – The total weighted number of people **aged 10-19** in a given state-year **based on their previous residence (`migplac1`)**.  
- **pred10num2** – The total weighted number of people **aged 20-29** in a given state-year **based on their previous residence**.  
- **pred10num3** – The total weighted number of people **aged 30-39** in a given state-year **based on their previous residence**.  
- **pred10num4** – The total weighted number of people **aged 40-49** in a given state-year **based on their previous residence**.  
- **pred10num5** – The total weighted number of people **aged 50-59** in a given state-year **based on their previous residence**.  
- **pred10num6** – The total weighted number of people **aged 60-69** in a given state-year **based on their previous residence**.  
- **pred10num7** – The total weighted number of people **aged 70-79** in a given state-year **based on their previous residence**.  
- **pred10num8** – The total weighted number of people **aged 80-89** in a given state-year **based on their previous residence**.  
- **pred10num9** – The total weighted number of people **aged 90+** in a given state-year **based on their previous residence**.  
- **pred10all** – The **total weighted population** across all age groups in a given state-year, based on individuals' previous residence.  

#### 3: From createBEA.do:

In [10]:
#3) from createBEA.do: 
gdpcomp = pd.read_stata(path_start + 'gdpandcomp.dta')

In [11]:
gdpcomp

,stfips,year,indcode,comp,gdp,IND1,IND2,IND3,IND4,IND5,...,yylag_IND12_2000,yylag_IND12_1990,yylag_IND13_2010,yylag_IND13_2000,yylag_IND13_1990,yylag_IND14_2010,yylag_IND14_2000,yylag_IND14_1990,region,cpi2010
0,1.0,1910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.266297
1,1.0,1920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.266297
2,1.0,1930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.266297
3,1.0,1940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.266297
4,1.0,1950,NaN,NaN,NaN,-1.756147,-3.698974,-3.085120,-1.713267,-2.914467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.266297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,56.0,1970,1.0,4470.266516,10114.258787,-2.443664,-2.356652,-2.776104,-2.804677,-2.864396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.266297
553,56.0,1980,1.0,9350.372217,21333.347817,-2.874111,-1.975046,-2.312600,-2.844203,-2.942833,...,0.000000,0.00000,0.0000,0.00000,0.000000,0.000000,0.00000,0.000000,4.0,1.266297
554,56.0,1990,1.0,7998.436732,19085.775276,-2.807389,-2.539000,-2.528977,-2.809109,-2.949421,...,0.000000,-5.11596,0.0000,0.00000,-1.613518,0.000000,0.00000,-2.797431,4.0,1.266297
555,56.0,2000,101.0,9859.387380,19916.317070,-3.118015,-2.673893,-2.414557,-2.873776,-2.938526,...,-4.431667,0.00000,0.0000,-1.36518,0.000000,0.000000,-2.53675,0.000000,4.0,1.266297


Explanation of column names for `gdpcomp`:

- **stfips** - Federal Information Processing Standards (FIPS) code, a unique two-digit numerical identifier assigned to each U.S. state and territory.  
- **year** - The specific year the data pertains to (**1969, 1979, 1989, 1999, or 2009**). 
- **indcode** - Denotes the industry code, which classifies industries according to NAICS (North American Industry Classification System).  - **`1` (early years) and `101` (later years)** likely correspond to **total private sector or aggregate GDP/compensation data**.  
- **comp** – **Total Compensation (millions of dollars)** (wages, salaries, and benefits) paid to employees in the state-year.  
- **gdp** – **Gross Domestic Product (GDP, millions of dollars) in current dollars** for the state-year-industry 
- **IND1** - Agriculture, Forestry, Fishing, Hunting  
- **IND2** - Mining (includes Oil and Gas Extraction)  
- **IND3** - Utilities  
- **IND4** - Construction and Manufacturing  
- **IND5** - Wholesale Trade  
- **IND6** - Retail Trade  
- **IND7** - Transportation and Warehousing  
- **IND8** - Information  
- **IND9** - Finance and Insurance  
- **IND10** - Real Estate and Rental Leasing  
- **IND11** - Professional, Scientific, and Technical Services  
- **IND12** - Educational Services  
- **IND13** - Health Care and Social Assistance  
- **IND14** - Arts, Entertainment, Recreation, Accommodation, and Food Services  




...more columns to explain


#### 4: ACS

In [ ]:
# ACS data
ACS = pd.read_stata(path_start + 'ACS.dta')

In [15]:
ACS

,year,sample,serial,cbserial,hhwt,cluster,statefip,strata,gq,pernum,...,educ,educd,empstat,empstatd,ind1990,wkswork1,wkswork2,uhrswork,incwage,migplac1
0,1930,1930 1%,1.0,NaN,100.95,1.930000e+12,Alabama,110100100.0,Households under 1970 definition,1,...,NaN,NaN,Employed,At work,NaN,NaN,NaN,NaN,NaN,NaN
1,1930,1930 1%,1.0,NaN,100.95,1.930000e+12,Alabama,110100100.0,Households under 1970 definition,2,...,NaN,NaN,Not in labor force,Not in Labor Force,NaN,NaN,NaN,NaN,NaN,NaN
2,1930,1930 1%,1.0,NaN,100.95,1.930000e+12,Alabama,110100100.0,Households under 1970 definition,3,...,NaN,NaN,Not in labor force,Not in Labor Force,NaN,NaN,NaN,NaN,NaN,NaN
3,1930,1930 1%,1.0,NaN,100.95,1.930000e+12,Alabama,110100100.0,Households under 1970 definition,4,...,NaN,NaN,Not in labor force,Not in Labor Force,NaN,NaN,NaN,NaN,NaN,NaN
4,1930,1930 1%,1.0,NaN,100.95,1.930000e+12,Alabama,110100100.0,Households under 1970 definition,5,...,NaN,NaN,Not in labor force,Not in Labor Force,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55455718,2011,2011 ACS,1366970.0,1501646.0,293.00,2.011014e+12,Wyoming,30056.0,Households under 1970 definition,3,...,Grade 11,Grade 11,Employed,At work,Department stores,NaN,50-52 weeks,40,22000.0,N/A
55455719,2011,2011 ACS,1366971.0,1501686.0,86.00,2.011014e+12,Wyoming,20056.0,Households under 1970 definition,1,...,Grade 12,Regular high school diploma,Employed,At work,Oil and gas extraction,NaN,50-52 weeks,50,100000.0,N/A
55455720,2011,2011 ACS,1366971.0,1501686.0,86.00,2.011014e+12,Wyoming,20056.0,Households under 1970 definition,2,...,Grade 12,Regular high school diploma,Not in labor force,Not in Labor Force,Child day care services,NaN,N/A,N/A,000000,N/A
55455721,2011,2011 ACS,1366972.0,1501931.0,40.00,2.011014e+12,Wyoming,30056.0,Households under 1970 definition,1,...,1 year of college,"1 or more years of college credit, no degree",Employed,At work,Colleges and universities,NaN,50-52 weeks,40,32000.0,N/A


Explanation of column names for `ACS`:
- **year** - Year survey was conducted  
- **sample** - Specific ACS sample identifier  
- **serial** - Unique household identifier within sample  
- **cbserial** - Census Bureau household identifier for internal use  
- **hhwt** - Household weight, used to produce population estimates  
- **cluster** - Geographic cluster or primary sampling unit for statistical weighting  
- **statefip** - Federal Information Processing Standard (FIPS) state code  
- **strata** - Survey stratum for variance estimation  
- **gq** - Indicates whether the person lives in a household (`1`) or in group quarters (`2` for institutions, `3` for non-institutional quarters like dorms)  
- **pernum** - Person number within the household (`1` householder, `2` spouse, etc.)  
- **perwt** - Survey weight for individuals, used to adjust sample representation  
- **sex** - Respondent's sex (`1 = Male`, `2 = Female`)  
- **age** - Respondent's age in years  
- **birthyr** - Respondent's birth year  
- **educ** - Education level in broad categories  
- **educd** - Detailed education category  
- **empstat** - Employment status  
- **empstatd** - More detailed employment status code  
- **ind1990** - Industry of employment based on 1990 Census industry codes  
- **wkswork1** - Number of weeks worked last year (broad categories)  
- **wkswork2** - Number of weeks worked last year (exact count)  
- **uhrswork** - Usual hours worked per week  
- **incwage** - Wage and salary income in nominal dollars  
- **migplac1** - Place of residence 1 year ago (coded by state or country) 